In [ ]:
# -- Cell 1 --

import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from io import StringIO

def get_stock_data(ticker: str) -> pd.DataFrame:
    data = yf.Ticker(ticker).history(period="max")
    data.reset_index(inplace=True)
    return data

def get_revenue_data(url: str) -> pd.DataFrame:
    headers = {"User-Agent": "Mozilla/5.0"}
    html_data = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html_data, "html.parser")
    tables = soup.find_all("table")
    if not tables:
        raise ValueError("No tables found. The website structure may have changed.")

    table_html = str(tables[0])
    revenue = pd.read_html(StringIO(table_html))[0]
    revenue.columns = ["Date", "Revenue"]
    revenue.dropna(inplace=True)
    revenue["Revenue"] = (
        revenue["Revenue"].str.replace(r"[$,]", "", regex=True)
        .astype(float)
    )
    revenue.dropna(inplace=True)
    return revenue

def make_dashboard(stock_df: pd.DataFrame, revenue_df: pd.DataFrame, title: str) -> None:
    fig, ax1 = plt.subplots(figsize=(12, 6))
    ax1.plot(stock_df["Date"], stock_df["Close"], color="tab:blue", label="Close")
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Stock Price ($)", color="tab:blue")
    ax1.tick_params(axis="y", labelcolor="tab:blue")

    ax2 = ax1.twinx()
    ax2.bar(revenue_df["Date"], revenue_df["Revenue"], alpha=0.3, color="tab:gray", label="Revenue")
    ax2.set_ylabel("Revenue (USD)", color="tab:gray")
    ax2.tick_params(axis="y", labelcolor="tab:gray")

    fig.suptitle(title)
    fig.tight_layout()
    plt.show()

In [ ]:
# Question 1: Use yfinance to Extract Tesla Stock Data
tesla_data = yf.Ticker("TSLA").history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()

In [ ]:
# Question 2: Use Webscraping to Extract Tesla Revenue Data
tesla_revenue_url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
tesla_revenue = get_revenue_data(tesla_revenue_url)
tesla_revenue["Date"] = pd.to_datetime(tesla_revenue["Date"])
tesla_revenue.tail()

In [ ]:
# Question 3: Use yfinance to Extract GameStop Stock Data
gme_data = yf.Ticker("GME").history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

In [ ]:
# Question 4: Use yfinance to Extract GameStop Revenue Data
gme_revenue_url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
gme_revenue = get_revenue_data(gme_revenue_url)
gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'])
gme_revenue.tail()

In [ ]:
# Question 5: Plot Tesla Stock Graph
make_dashboard(tesla_data, tesla_revenue, "Tesla Stock Price & Revenue")

In [ ]:
# Question 6: Plot GameStop Stock Graph
make_dashboard(gme_data, gme_revenue, "GameStop Stock Price & Revenue")